# Build YOLO model

In [1]:
from keras.layers import Input, Conv2D, Add, ZeroPadding2D, UpSampling2D, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2

d:\ProgramData\Anaconda2\envs\py3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Darknet53

In [2]:
def conv_unit(x, filters, kernels, padding='same', strides=1):
    x = Conv2D(filters, kernels,
               padding=padding,
               strides=strides,
               use_bias=False,
               kernel_regularizer=l2(5e-4))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    return x

In [3]:
def residual_block(inputs, filters, n):
    x = ZeroPadding2D(((1, 0),(1, 0)))(inputs)
    x = conv_unit(x, filters, (3, 3), padding='valid', strides=2)
    for i in range(n):
        y = conv_unit(x, filters//2, (1, 1))
        y = conv_unit(y, filters, (3, 3))
        x = Add()([x, y])

    return x

In [4]:
def darknet53(inputs):
    x = conv_unit(inputs, 32, (3, 3))
    x = residual_block(x, 64, 1)
    x = residual_block(x, 128, 2)
    x = residual_block(x, 256, 8)
    x = residual_block(x, 512, 8)
    x = residual_block(x, 1024, 4)
    
    return x

## YOLOv3

In [5]:
def top_conv(x, filters, out_dim):
    x = conv_unit(x, filters//2, (1, 1))
    x = conv_unit(x, filters, (3, 3))
    x = conv_unit(x, filters//2, (1, 1))
    x = conv_unit(x, filters, (3, 3))
    x = conv_unit(x, filters//2, (1, 1))
    
    y = conv_unit(x, filters, (3, 3))
    y = Conv2D(out_dim, (1, 1),
               padding='same',
               kernel_regularizer=l2(5e-4))(y)
    
    return x, y

In [6]:
def yolo_model(inputs, num_anchors, num_classes):
    darknet = Model(inputs, darknet53(inputs))
    x, y1 = top_conv(darknet.output, 1024, num_anchors*(num_classes+5))
    
    x = conv_unit(x, 256, (1, 1))
    x = UpSampling2D(2)(x)
    x = Concatenate()([x, darknet.layers[152].output])
    x, y2 = top_conv(x, 512, num_anchors*(num_classes+5))
    
    x = conv_unit(x, 128, (1, 1))
    x = UpSampling2D(2)(x)
    x = Concatenate()([x, darknet.layers[92].output])
    x, y3 = top_conv(x, 256, num_anchors*(num_classes+5))
    
    return Model(inputs, [y1,y2,y3])
    

## Build model

In [7]:
inputs = Input((416, 416, 3))
model = yolo_model(inputs, 3, 80)

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 416, 416, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 416, 416, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 416, 416, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
zero_paddi

In [9]:
model.load_weights('yolo.bak.h5')

In [10]:
model.save('yolo.h5')

## Save notop model for training

In [11]:
base_model = Model(model.input, [model.layers[-4].output, model.layers[-5].output, model.layers[-6].output])
base_model.save('yolo_base.h5')
del base_model